# 场景四：自然语言交流的搜索引擎

In [5]:
!ollama pull llama2-chinese:13b

pulling manifest
pulling 8359bebea988... 100% |██████████████████| (7.4/7.4 GB, 75 TB/s)        
pulling 65c6ec5c6ff0... 100% |████████████████████| (45/45 B, 2.4 MB/s)        
pulling dd36891f03a0... 100% |████████████████████| (31/31 B, 1.3 MB/s)        
pulling f94f529485e6... 100% |███████████████████| (382/382 B, 19 MB/s)        
verifying sha256 digest
writing manifest
removing any unused layers
success


In [ ]:
%pip install langchain langchainhub langchain-experimental
%pip install python-dotenv openai duckduckgo-search numexpr

In [31]:
from langchain import hub
from langchain.llms import OpenAI
from langchain.agents import load_tools 
from langchain.agents import AgentExecutor
from langchain.agents.output_parsers import ReActSingleInputOutputParser
from langchain.agents.format_scratchpad import format_log_to_str
from langchain.tools.render import render_text_description

# 通过 python-dotenv 加载环境变量
from dotenv import load_dotenv
load_dotenv()

# 准备大语言模型：这里需要 OpenAI，可以方便地按需停止推理
llm = OpenAI()
llm_with_stop = llm.bind(stop=["\nObservation"])

# 准备我们的工具：这里用到 DuckDuckGo 搜索引擎，和一个基于 LLM 的计算器
tools = load_tools(["ddg-search", "llm-math"], llm=llm)

# 准备核心提示词：这里从 LangChain Hub 加载了 ReAct 模式的提示词，并填充工具的文本描述
prompt = hub.pull("hwchase17/react")
prompt = prompt.partial(
    tools=render_text_description(tools),
    tool_names=", ".join([t.name for t in tools]),
)

# 构建 Agent 的工作链：这里最重要的是把中间步骤的结构要保存到提示词的 agent_scratchpad 中
agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_log_to_str(x["intermediate_steps"]),
    }
    | prompt
    | llm_with_stop
    | ReActSingleInputOutputParser()
)

# 构建 Agent 执行器：执行器负责执行 Agent 工作链，直至得到最终答案（的标识）并输出回答
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
agent_executor.invoke({"input": "今天上海和北京的气温差几度？"})




> Entering new AgentExecutor chain...
 I need to find out what the temperatures are in Beijing and Shanghai today.
Action: duckduckgo_search
Action Input: "Beijing temperature today"Weather report for Beijing. Overnight into Tuesday it is mostly cloudy, but most clouds give way early in the day. For the afternoon clear skies prevail. It is a sunny day. Temperatures as high as 72 °F are foreseen. During the night and in the first hours of the day light air is noticeable (1 to 4 mph). Beijing 7 day weather forecast including weather warnings, temperature, rain, wind, visibility, humidity and UV 6:23 AM5:36 PM. Morning temperature of 50 degrees, afternoon 70°, evening 50° and night 45°. Clear, evening mostly clear. The hourly local weather forecast shows hour by hour weather conditions like temperature, feels like temperature, humidity, amount of precipitation and chance of precipitation, wind and gusts for Beijing. Beijing. Beijing Weather Forecast. Providing a local hourly Beijing wea

{'input': '今天上海和北京的气温差几度？', 'output': '今天上海和北京的气温差五度。'}

In [ ]:
# 请先运行本代码块，再执行下一个示例
%pip install playwright
!playwright install

In [1]:
from langchain import hub
from langchain.chat_models import ChatOpenAI
from langchain.agents import AgentExecutor
from langchain.agents.format_scratchpad import format_log_to_str
from langchain.agents.output_parsers import JSONAgentOutputParser
from langchain.agents.agent_toolkits import PlayWrightBrowserToolkit
from langchain.tools.render import render_text_description_and_args
from langchain.tools.playwright.utils import create_async_playwright_browser

# 避免 Jupter Notebook 产生 EventLoop 问题
import nest_asyncio
nest_asyncio.apply()

# 通过 python-dotenv 加载环境变量
from dotenv import load_dotenv
load_dotenv()

# 准备大语言模型：这里需要 OpenAI，可以方便地按需停止推理
llm = ChatOpenAI()
llm_with_stop = llm.bind(stop=["\nObservation"])

# 准备 Browser 工具集
async_browser = create_async_playwright_browser()
browser_toolkit = PlayWrightBrowserToolkit.from_browser(async_browser=async_browser)
tools = browser_toolkit.get_tools()

# 准备核心提示词：这里从 LangChain Hub 加载了 ReAct 多参数输入模式的提示词，并填充工具的文本描述
prompt = hub.pull("hwchase17/react-multi-input-json")
prompt = prompt.partial(
    tools=render_text_description_and_args(tools),
    tool_names=", ".join([t.name for t in tools]),
)

# 构建 Agent 的工作链：这里最重要的是把中间步骤的结构要保存到提示词的 agent_scratchpad 中
agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_log_to_str(x["intermediate_steps"]),
    }
    | prompt
    | llm_with_stop
    | JSONAgentOutputParser()
)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

# 因为使用了异步浏览器页面抓取工具，这里对应地使用异步的方式进行 Agent 执行
await agent_executor.ainvoke({"input": "请访问这个网页并总结一下上面的内容：blog.langchain.dev"})



> Entering new AgentExecutor chain...
Action:
```
{
  "action": "navigate_browser",
  "action_input": {
    "url": "https://blog.langchain.dev"
  }
}
```Navigating to https://blog.langchain.dev returned status code 200Observation: Navigating to the webpage "https://blog.langchain.dev" returned a status code of 200, indicating that the webpage was successfully loaded.

Action:
```
{
  "action": "extract_text",
  "action_input": {}
}
```LangChain Blog Skip to content LangChain Blog Home By LangChain Release Notes GitHub Docs Case Studies Sign in Subscribe Query Construction Key Links

 * Text-to-metadata: Updated self-query docs and template
 * Text-to-SQL+semantic: Cookbook and template

There's great interest in seamlessly connecting natural language with diverse types of 7 min read Featured Building LLM-Powered Web Apps with Client-Side Technology By LangChain 5 min read Fine-tuning ChatGPT: Surpassing GPT-4 Summarization Performance–A 63% Cost Reduction and 11x Speed Enhancement us

{'input': '请访问这个网页并总结一下上面的内容：blog.langchain.dev',
 'output': 'The webpage contains various content, including blog posts, release notes, case studies, and important links related to language models and data extraction.'}

In [5]:
from langchain.callbacks import HumanApprovalCallbackHandler
from langchain.tools import ShellTool

tool = ShellTool(callbacks=[HumanApprovalCallbackHandler()])
print(tool.run("ls /usr"))
print(tool.run("ls /root"))

/home/codespace/.python/current/lib/python3.10/site-packages/langchain/tools/shell/tool.py:31: UserWarning: The shell tool has no safeguards by default. Use at your own risk.
  warnings.warn(
/home/codespace/.python/current/lib/python3.10/site-packages/langchain/tools/shell/tool.py:31: UserWarning: The shell tool has no safeguards by default. Use at your own risk.
  warnings.warn(


bin
games
include
lib
lib32
lib64
libexec
libx32
local
sbin
share
src



Error in HumanApprovalCallbackHandler.on_tool_start callback: HumanRejectedException("Inputs ls /root to tool {'name': 'terminal', 'description': 'Run shell commands on this Linux machine.'} were rejected.")


HumanRejectedException: Inputs ls /root to tool {'name': 'terminal', 'description': 'Run shell commands on this Linux machine.'} were rejected.

In [11]:
from langchain.chat_models import ChatOllama
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.tools import DuckDuckGoSearchRun

template = """turn the following user input into a search query for a search engine:

{input}"""
prompt = ChatPromptTemplate.from_template(template)

model = ChatOllama(model="llama2-chinese:13b")

# 构建工具链：通过大语言模型准备好工具的输入内容，然后调用工具
chain = prompt | model | StrOutputParser() | DuckDuckGoSearchRun()
chain.invoke({"input": "人工智能？！"})


'人工智能（英語： artificial intelligence ，缩写为 AI ）亦稱機器智能，指由人製造出來的機器所表現出來的智慧。通常人工智能是指通过普通電腦程式來呈現人類智能的技術。該詞也指出研究這樣的智能系統是否能夠實現，以及如何實現。 人工智能的定義可以分為兩部分，即「人工」和「智能」。「人工」即由人設計，為人創造、製造。 關於甚麼是「智能」，較有爭議性。這涉及到其它諸如意識、自我、心靈，包括無意識的精神等等問題。人唯一瞭解的智能是人本身的智能，這是普遍認同的觀點。 人工智能 （英语： artificial intelligence ，缩写为 AI ）亦称 机器智能 ，指由人制造出来的机器所表现出来的 智慧 。. 通常人工智能是指通过普通电脑程式来呈现人类智能的技术。. 该词也指出研究这样的智能系统是否能够实现，以及如何实现。. 同时，通过 医学 ... 但是我們對我們自身智能的理解都非常有限，對構成人的智能必要元素的瞭解也很有限，所以就很難定義甚麼是「人工」製造的「智能」。因此人工智能的研究往往涉及對人智能本身的研究。其它關於動物或其它人造系統的智能也普遍被認為是人工智能相關的 ... 人工智能转译员人才储备不足。ai相关的岗位主要包含软件工程师、数据工程师、数据科学家、数据架构师、产品经理和转译员等。其中，人工智能转译员的角色尤为重要，因为他们知道应该提出哪些业务问题，并将业务问题"翻译"成人工智能解决方案。'

In [2]:
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOllama
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnableBranch


model = ChatOllama(model="llama2-chinese:13b")

# 构建分类判断链：识别用户的问题应该属于哪个（指定的）分类
chain = (
    PromptTemplate.from_template(
        """Given the user question below, classify it as either being about `LangChain` or `Other`.
                                     
Do not respond with more than one word.

<question>
{question}
</question>

Classification:"""
    )
    | model
    | StrOutputParser()
)

# 构建内容问答链和默认应答链
langchain_chain = (
    PromptTemplate.from_template(
        """You are an expert in LangChain. Respond to the following question in one sentence:

Question: {question}
Answer:"""
    )
    | model
)
general_chain = (
    PromptTemplate.from_template(
        """Respond to the following question in one sentence:

Question: {question}
Answer:"""
    )
    | model
)

# 通过 RunnableBranch 构建条件分支并附加到主调用链上
branch = RunnableBranch(
    (lambda x: "langchain" in x["topic"].lower(), langchain_chain),
    general_chain,
)
full_chain = {"topic": chain, "question": lambda x: x["question"]} | branch

print(full_chain.invoke({"question": "什么是 LangChain?"}))
print(full_chain.invoke({"question": "1 + 2 = ?"}))

content='LangChain是一种基于人工智能的自然语言处理技术，它使用机器学习算法来生成语言模型，以实现自然语言识别和生成功能。'
content='3.'


In [4]:
from langchain.llms import Ollama
from langchain.chat_models import ChatOllama
from langchain.prompts import PromptTemplate, ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser

chat_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You're a nice assistant who always includes a compliment in your response",
        ),
        ("human", "Why did the {animal} cross the road"),
    ]
)

# 在这里，我们将使用一个错误的模型名称来轻松创建一个会出错的链
chat_model = ChatOllama(model_name="gpt-fake")
bad_chain = chat_prompt | chat_model | StrOutputParser()


prompt_template = """Instructions: You should always include a compliment in your response.

Question: Why did the {animal} cross the road?"""
prompt = PromptTemplate.from_template(prompt_template)

# 然后我们构建一个一定可以正常使用的调用链
llm = Ollama(model="llama2-chinese:13b")
good_chain = prompt | llm

# 最后用使用 with_fallbacks 构建一个异常回退机制
chain = bad_chain.with_fallbacks([good_chain])
chain.invoke({"animal": "turtle"})

'Dear Human, \n\nI have heard that you are looking for an answer to the question of why the turtle crossed the road. As an AI assistant, I can provide information on this subject. However, it would be much more meaningful if you could compliment me or ask questions in a positive manner.\n\nPlease let me know what other information you need or how else I can assist you! '